<div style="
  padding: 5pt;
  border-style: solid;
  border-width: 1px;
  border-color: gray;
  border-radius: 10px;">

# **Python et intelligence artificielle**

# *Séance n°8 : Optimisation et descente de gradient*

</div>

## Objectif

Lors de la séance précédente, vous avez mis en oeuvre une régression linéaire à l'aide de la classe `LinearRegression` de *Scikit-learn* en entraînant le modèle à l'aide de sa méthode `fit()`. Aujourd'hui, nous allons explorer en détail le mécanisme sous-jacent : comment les paramètres d'un modèle de régression linéaire sont-ils ajustés pour minimiser l'erreur de prédiction ?

L'objectif de cette séance est d'introduire et de mettre en pratique la **descente de gradient**, une méthode d'optimisation fondamentale en apprentissage automatique. Vous allez notamment :

- Comprendre la notion de **fonction de coût** dans le cadre de la régression linéaire.
- Implémenter l'algorithme de **descente de gradient** afin de minimiser cette fonction.
- Comparer trois variantes de l'algorithme : **par lot**, **stochastique** et par **mini-lots**.
- Observer l'effet de la **standardisation des données** sur la convergence de la descente de gradient.
- Appliquer la descente de gradient à un problème d'estimation de résistances.

## Introduction

### Régression linéaire

On rappelle qu'une régression linéaire simple cherche à modéliser la relation entre une variable cible $y$ et une variable explicative $X$. Cette relation est exprimée sous la forme d'une droite :

$$
y = \theta_1 X + \theta_0
$$

où :

- $\theta_0$ est l'ordonnée à l'origine, soit la valeur de $y$ lorsque $X = 0$.
- $\theta_1$ représente la pente de la droite et indique la variation de $y$ pour chaque unité d'augmentation de $X$.

### Fonction de coût

En apprentissage automatique, on cherche à minimiser l'erreur entre les prédictions du modèle et les valeurs réelles. Cette erreur est mesurée par une fonction de coût. Pour la régression linéaire, la fonction de coût la plus courante est l'erreur quadratique moyenne :

$$
J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right)^2
$$

où :

- $m$ est le nombre d'exemples dans le jeu de données ;
- $h_{\theta}(x^{(i)})$ est la prédiction pour l'exemple $i$ avec les paramètres $\theta$ ;
- $y^{(i)}$ est la valeur réelle pour l'exemple $i$.

L'objectif est de minimiser cette fonction en trouvant les paramètres $\theta_0$ et $\theta_1$.

#### Nécessité d'une colonne de biais pour $X$

La fonction de prédiction est la suivante :

$$
h_{\theta}(X) = X \cdot \theta
$$

Si $X$ est une matrice sans colonne de biais, elle contiendra uniquement les valeurs de la variable explicative (ex : température, taille, etc.). Cependant, pour inclure $\theta_0$, nous devons ajouter à chaque ligne de $X$ une première colonne remplie de 1 de manière à tenir compte du décalage constant à chaque prédiction.

En Python, on utilise souvent `np.c_` pour concaténer une colonne de 1 à $X$ de la manière suivante :

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
import numpy as np

X = np.array([[1], [2], [3]])
X_biais = np.c_[np.ones((X.shape[0], 1)), X]
print(X_biais)
```
</div>

Une autre solution consiste à faire appel à la fonction `hstack()` de *Numpy* :

<div style="
  padding: 5pt;
  border-style: dashed;
  border-width: 1px;
  border-color: gray;">

```python
import numpy as np

m = len(y)
# Ajout d'une colonne de 1 à X pour le biais
X_biais = np.hstack((np.ones((m, 1)), X))
```

</div>

### Descente de gradient

La descente de gradient est une méthode pour ajuster les paramètres $\theta$ afin de minimiser la fonction de coût. Elle consiste à calculer, pour chaque paramètre, dans quelle direction ajuster sa valeur de manière à réduire la fonction de coût. À chaque itération, l'algorithme ajuste les paramètres dans cette direction en suivant la règle :

$$
\theta := \theta - \alpha \nabla_{\theta} J(\theta)
$$

où :

- $\alpha$ est le taux d'apprentissage (*learning rate*) qui contrôle l'amplitude des ajustements. S'il est trop élevé, l'algorithme risque de diverger. S'il est trop faible, sa convergence sera lente.
- $\nabla_{\theta} J(\theta)$ est le gradient de la fonction de coût par rapport aux paramètres $\theta$.

Le gradient, en somme, est un vecteur qui pointe dans la direction de la plus forte augmentation de la fonction de coût. En prenant la direction opposée, on réduit la valeur de la fonction de coût.
Imaginez que vous descendiez une colline à l'aveugle. À chaque étape, vous ajustez votre trajectoire pour descendre dans la direction la plus raide que vous ressentez. C'est ce que fait la descente de gradient : elle "descend" le long de la fonction de coût afin de trouver son minimum.

### Calcul du gradient dans le cas de la régression linéaire

Dans le cas d'une régression linéaire, la fonction de coût a une forme quadratique, ce qui simplifie le calcul du gradient. Le gradient est donné par :

$$
\nabla_{\theta} J(\theta) = \frac{1}{m} X^T (X \theta - y)
$$

Nous utiliserons cette formule pour ajuster les paramètres $\theta$ à chaque itération et ainsi minimiser la fonction de coût.

### Variantes de la descente de gradient

Il existe plusieurs variantes de la descente de gradient, chacune ayant des propriétés spécifiques qui influencent le temps de convergence et la précision. En voici trois :

1. La **descente par lot** (*batch gradient descent*) qui utilise l'ensemble des données pour calculer le gradient à chaque itération. Ell converge de manière stable vers le minimum global (surtout pour les fonctions convexes comme ici), par contre l'algorithme est long à exécuter sur de très grands jeux de données, car chaque itération nécessite l'ensemble des exemples.
2. La **descente stochastique** (*stochastic gradient descent*) qui utilise un seul exemple aléatoire à chaque mise à jour des paramètres. Elle converge rapidement, même pour de grands ensembles de données, mais le chemin de convergence peut être irrégulier et oscillant autour du minimum.
3. La **descente par mini-lots** (*mini-batch gradient descent*) qui divise le jeu de données en mini-lots d'exemples pour calculer les mises à jour. C'est un compromis entre la vitesse de la descente stochastique et la stabilité de la descente par lot. Par contre, elle nécessite de bien choisir la taille des mini-lots pour un compromis optimal.

Pour un jeu de données contenant des milliers de points, la descente stochastique peut converger plus rapidement que la descente par lot complet, mais au prix d'une précision réduite. Par ailleurs, les mini-lots permettent d'obtenir une convergence stable avec un compromis acceptable entre vitesse et précision.

Remarque : les variables dans les jeux de données peuvent avoir des échelles différentes (par exemple, température en °C et pression en hPa). La standardisation (centrage autour de 0 et réduction à un écart-type de 1) peut améliorer la vitesse de convergence en ajustant toutes les variables à la même échelle.

---

## Exercices



### Exercice 1 : implémentation de la fonction de coût

Avant d'implémenter la descente de gradient, nous devons d'abord écrire la fonction de coût, qui est un critère de performance du modèle. La fonction de coût à utiliser est l'erreur quadratique moyenne (ou MSE).

1. Écrivez la fonction `calculer_cout(X, y, theta)` qui retourne l'erreur quadratique moyenne dans le cas d'une régression linéaire. Les paramètres de la fonction sont :
   - `X` : une matrice représentant les valeurs de la variable explicative.
   - `y` : un vecteur des valeurs cibles.
   - `theta` : un vecteur des paramètres du modèle $(\theta_0, \theta_1)$.
2. Testez la fonction avec les valeurs suivantes et vérifiez qu'elle vous retourne environ 2,33.

   <div style="
      padding: 5pt;
      border-style: dashed;
      border-width: 1px;
      border-color: gray;">

   ```python
   X = np.array([[1], [2], [3]])
   y = np.array([[2], [4], [6]])
   theta = np.array([[0], [1]])
   ```

   </div>

#### Solution


In [ ]:
# Votre code ici



---

### Exercice 2 : Implémentation de la descente de gradient

1. Écrivez la fonction `descente_gradient(X, y, theta, alpha, n_iterations)` permettant de minimiser la fonction de coût et d'ajuster les paramètres du modèle. Les paramètres de la fonction sont :

   - `X` : une matrice représentant les valeurs de la variable explicative **et sa colonne de biais**.
   - `y` : un vecteur des valeurs cibles.
   - `theta` : un vecteur des paramètres du modèle $(\theta_0, \theta_1)$.
   - `alpha` : le taux d'apprentissage.
   - `n_iterations` : le nombre d'itérations de l'algorithme.

   La fonction retourne `theta` ainsi qu'une liste de l'historique des coûts.
2. Tracez l'évolution du coût

#### Solution


In [ ]:
# Votre code ici



---

### Exercice 3 : Expérience de mesure de résistance

Dans cet exercice, vous allez appliquer l'algorithme de descente de gradient pour modéliser la résistance $R$ d'un matériau en fonction de la température $T$. On considère une relation linéaire entre $R$ et $T$ de la forme :

$$
R = \theta_0 + \theta_1 T
$$

1. Utilisez le code ci-dessous pour générer un ensemble de données de température et de résistance simulées avec un bruit aléatoire.
   
   <div style="
      padding: 5pt;
      border-style: dashed;
      border-width: 1px;
      border-color: gray;">

   ```python
   import numpy as np
   import matplotlib.pyplot as plt

   # Paramètres réels
   R0_true = 10  # Ohms
   alpha_true = 0.004  # 1/°C

   # Génération des températures et résistances
   T = np.linspace(0, 100, 50).reshape(-1, 1)
   noise = np.random.normal(0, 0.5, T.shape)
   R = R0_true * (1 + alpha_true * T) + noise

   plt.scatter(T, R)
   plt.xlabel('Température (°C)')
   plt.ylabel('Résistance (Ohms)')
   plt.show()
   ```

   </div>

2. Standardisez les données pour que les différences d'échelle ne ralentisse pas la convergence.
3. Utilisez la fonction de descente de gradient que vous avez implémentée pour estimer les valeurs de $\theta_0$ et $\theta_1$.
4. Affichez la courbe de convergence de la fonction de coût.
5. Comparez les paramètres estimés avec les valeurs réelles.
6. Tracez les prédictions du modèle sur les données et commentez la qualité de l'ajustement.

#### Solution


In [ ]:
# Votre code ici



---

### Exercice 4 : Impact de la standardisation et comparaison des méthodes

Dans cet exercice, vous allez explorer comment la standardisation et les variantes de la descente de gradient influencent la convergence.

1. Répétez l'exercice précédent sans standardiser les données et observez les différences dans la convergence.
2. Commentez les effets observés.
3. Reprenez l'implémentation de `descente_gradient()` et adaptez-la pour chaque méthode.
4. Pour chaque méthode, visualisez la convergence et comparez le nombre d'itérations nécessaires pour atteindre une bonne précision.
5. Expliquez comment chaque méthode influence la précision et la vitesse de convergence.
6. Indiquez dans quels contextes chaque méthode serait à privilégier.

---

## Conclusion

Dans cette séance vous avez :

- mis en pratique la descente de gradient dans le cas d'un modèle de régression linéaire ;
- comparé l'impact de la standardisation sur la convergence ;
- observé comment chaque variante de descente de gradient affecte la vitesse et la précision de l'algorithme.

